## Extracting genomic features

In [ ]:
from Bio import SeqIO
from Bio.SeqUtils import GC
from Bio.SeqIO.FastaIO import SimpleFastaParser
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
# Initializing lists in order to extract feature information from viral sequences
sequence_lengths=[]
sequence_gc=[]
seq_ids=[]
sequence_adenine=[]
sequence_guanine=[]
sequence_cytosine=[]
sequence_thymine=[]

In [ ]:
# Parsing features from viral sequences - an id to denote where the sequence came from, the genome length, and the GC content
with open('./GOV2_viral_populations_larger_than_5KB_or_circular.fasta') as file:
    for header,sequence in SimpleFastaParser(file):
        sequence_lengths.append(len(sequence))
        seq_ids.append(header)
        sequence_gc.append(GC(sequence))
        sequence_adenine.append(sequence.count('A'))
        sequence_guanine.append(sequence.count('G'))
        sequence_cytosine.append(sequence.count('C'))
        sequence_thymine.append(sequence.count('T'))

In [ ]:
################################
# Initializing environmental data categories to parse from sequence headers
station_id=[]
depth_class=[]
for line in seq_ids:
    if len(line)!=0:
        station_id.append(line.split('_')[0])
        depth_class.append(line.split('_')[1])
    else:
        station_id.append('NA')
        depth_class.append('NA')
        print('No Seq Header')

In [ ]:
data=np.array([seq_ids,sequence_lengths,sequence_gc,sequence_adenine,sequence_thymine,sequence_guanine,sequence_cytosine,station_id,depth_class])
data=data.transpose()
sequence_stats=pd.DataFrame(data,columns=['ID','length','GC','A','T','G','C','station','depth_class'])
no_malaspina=sequence_stats[~sequence_stats.depth_class.isin(['Malaspina','NA'])]
no_malaspina['depth_class'].replace('IZZ','ZZZ')
genome_covariates = no_malaspina

In [ ]:
genome_covariates['length']=genome_covariates['length'].apply(float)
genome_covariates['GC']=genome_covariates['GC'].apply(float)
genome_covariates['A']=genome_covariates['A'].apply(float)
genome_covariates['T']=genome_covariates['T'].apply(float)
genome_covariates['C']=genome_covariates['C'].apply(float)
genome_covariates['G']=genome_covariates['G'].apply(float)

In [ ]:
# The elemental formulae for the nucleotides are as follows:
# adenine: C5H5N5
# guanine: C5H5N5O
# cytosine: C4H5N3O
# thymine: C5H6N2O2
# We will convert the nucleotide frequencies to elemental C:N content and ratios
# Assuming double stranded DNA viruses
genome_covariates=genome_covariates.assign(total_c=2*(genome_covariates['A']*5+genome_covariates['G']*5+genome_covariates['C']*4+genome_covariates['T']*5+5))
genome_covariates=genome_covariates.assign(total_n=2*(genome_covariates['A']*5+genome_covariates['G']*5+genome_covariates['C']*3+genome_covariates['T']*2))
genome_covariates=genome_covariates.assign(c_to_n=genome_covariates['total_c']/genome_covariates['total_n'])

In [ ]:
genome_covariates.to_csv('genome_covariates.csv', index=False)